In [1]:
import sys
print(sys.version)
if sys.version[:3] not in ["3.8", "3.9"]:
    print("You need to use a conda environment or upgrade your Python version to at least 3.8")

3.8.11 (default, Aug  6 2021, 08:56:27) 
[Clang 10.0.0 ]


In [2]:
import warnings
warnings.filterwarnings('ignore')

import math

import cloudpickle
import numpy as np
import skopt

ModuleNotFoundError: No module named 'cloudpickle'

In Lab 1, you gained intuition for how to optimize the parameters for expensive-to-evaluate functions using Bayesian optimization. Bayesian optimiation is a powerful tool, and there are lots of open-source Bayesian optimization libraries, so you will rarely need to implement the low-level details of Bayesian optimization yourself. However, as you will see during this lab, applying Bayesian optimization to real-world problems is rarely as simple as making a single call to a library function.

In this lab, imagine that you are in charge of administering a multi-tenant cloud machine that is storing data from ten different customers (let's call them customer 1 through customer 10). Each customer stores 1 GB of data, so the cloud machine stores 10 GB of total data on its disk. Each customer runs queries over their own data, which requires reading the relevant data from disk into memory for processing. As a concrete example, imagine that customer 1 is MIT, and MIT is storing data about all undergraduate students on the cloud machine's disk, and MIT runs queries such as "what is the average GPA of course 6 undergrads?", which requires reading a portion of the data (only the portion regarding course 6 undergrads) into memory for processing.

The cloud machine has enough memory to cache a total of 1 GB of data. Cached data is pinned in memory and is therefore much faster to process than data that must be read from disk. Ideally, we want all data to be cached in memory, but in this case there's only enough cache space to store 10% of the total data in cache.

To preserve customer privacy, you are not allowed to share the cache among the ten customers. Therefore, you must decide how to allocate the 1 GB cache budget among the 10 customers. More concretely, you must define a vector $x = [x_1, x_2, \ldots, x_{10}]$, where $x_i$ is the amount of cache given to customer $i$'s data, measured in bytes.

How do you find the best setting for $x$? First, you need a way to evaluate the performance of a given setting for $x$. For privacy reasons, you are not allowed to directly observe each customer's query workload. All you can do is set $x$, wait for a while, and measure the resulting performance. Therefore, evaluating the performance for a given $x$ is expensive.  This seems like a perfect fit for Bayesian optimization.

# Assignment 0: Setup

Let's say we want to measure performance in terms of the average time it takes to process a query. (In the real world, depending on customer expectations and business requirements, you might measure performance a different way, e.g., in terms of tail latencies or cache hits.)

For this lab, we will provide you with a Python function, `measure_average_query_time`. It takes a single argument, which is the vector $x$ (represented as a Python list), and outputs the time in seconds that it takes to run an average query across all customers. In reality, this function would take a while to run (because we need to wait for customer queries to actually run), but for the sake of this lab, the provided function will run very quickly.

In [6]:
# Load the Python function
with open("measure_average_query_time.bin", "rb") as f:
    measure_average_query_time = cloudpickle.loads(f.read())

For example, if we allocate 100 MB to each customer's cache, the average query time would be:

In [7]:
measure_average_query_time([int(1e8)] * 10)

6.890539902693893

In this lab, we will be using the scikit-optimize library, which implements Bayesian optimization. You can find documentation and examples [here](https://scikit-optimize.github.io/stable/index.html).

Let's try using scikit-optimize to find the best setting for $x$. (This may run for about half a minute.)

In [ ]:
# An objective function which returns the average query time achieved with a certain
# setting of cache sizes
def objective(cache_sizes):
    return measure_average_query_time(cache_sizes)

In [ ]:
res = skopt.gp_minimize(         # Bayesian optimization using Gaussian processes
    objective,                   # The function to minimize
    [(0, int(1e9))] * 10,        # The bounds on each dimension of x (we should not allocate more than 1 GB to any customer's cache)
    n_calls=50,                  # Total number of samples to collect
)  

The best setting for $x$ that the optimizer found can be accessed through `res.x`.

In [ ]:
res.x

For most customers, the optimizer allocated 1 GB of cache. Obviously, this would minimize average query time, but it's not a valid setting, because we only have 1 GB of total cache, so this result is useless. This brings us to the first challenge: how do we incorporate the constraint that the total cache size cannot be more than 1 GB into the optimization?

# Assignment 1: Incorporating constraints

The total size of all caches must be no more than 1 GB. There are many possible ways to incorporate this constraint into the optimization problem. Here, we explore two of those ways.

#### Assignment 1a

The first way is to incorporate the constraint into the objective function directly. Think of a way to modify the objective function by penalizing any setting of $x$ that sums to more than 1 GB.

In [ ]:
max_total_cache_size = 1e9

In [ ]:
def objective_v1(cache_sizes):
    # Insert code here that penalizes settings where the total size of all caches is more than 1 GB.
    # Hint: you may need to scale the penalty based on the amount by which the maximum size is exceeded.

Now if we run the Bayesian optimization, the sum of $x$ should be no more than 1 GB.

In [ ]:
res = skopt.gp_minimize(
    objective_v1,
    [(0, int(1e9))] * 10,
    n_calls=50,
)
print(f"Total cache size: {sum(res.x)}, avg query time achieved: {res.fun}")
if int(sum(res.x)) > max_total_cache_size:
    print("Oh no! Total cache size constraint violated. Something went wrong.")

#### Assignment 1b

The second way is to reinterpret $x$. Instead of interpreting it as the exact cache sizes per customer in bytes, think of them as relative cache sizes. Can you scale $x$ so that the total cache size is no more than 1GB?

In [ ]:
def compute_scaled_cache_sizes(cache_sizes):
    # Insert code here that scales `cache_sizes` so that it sums to no more than 1 GB,
    # and returns the scaled version.

In [ ]:
def objective_v2(cache_sizes):
    scaled_cache_sizes = compute_scaled_cache_sizes(cache_sizes)
    return measure_average_query_time(scaled_cache_sizes)

Now if we run the Bayesian optimization, the sum of scaled $x$ should be no more than 1 GB.

In [ ]:
res = skopt.gp_minimize(
    objective_v2,
    [(0, int(1e9))] * 10,
    n_calls=50,
)
best_cache_sizes = compute_scaled_cache_sizes(res.x)
print(f"Total cache size: {sum(best_cache_sizes)}, avg query time achieved: {objective_v2(best_cache_sizes)}")
if int(sum(best_cache_sizes)) > max_total_cache_size:
    print("Oh no! Total cache size constraint violated. Something went wrong.")

#### Assignment 1c

For each of these two methods, describe one advantage that it has over the other method.

Your text here:

For the remainder of this lab, we will continue to use the second method above.

In [ ]:
def objective(cache_sizes):
    return objective_v2(cache_sizes)

# Assignment 2: Multi-objective optimization

So far we have set $x$ in a way that minimizes average query time across all customers. But what if customer 1 is a very important customer, and they make a request that their own average query time does not fall below a certain value $T$? Therefore, we want to minimize the average query runtime across all customers while guaranteeing that average runtime for customer 1 is no more than $T$. For this assignment, we will set $T$ to 7.4:

In [ ]:
desired_cust1_avg_time = 7.4

Now how do we find the best $x$? Again, there are multiple viable methods. One method is to do something similar to Assignment 1a, where we incorporated the constraint into the optimization objective. Here, we explore a different method.

First, we provide you with a Python function `measure_average_query_time_customer1`, which takes a single value as input (the cache size for customer 1, in bytes) and returns customer 1's average query time.

In [ ]:
with open("measure_average_query_time_customer1.bin", "rb") as f:
    measure_average_query_time_customer1 = cloudpickle.loads(f.read())

In [ ]:
measure_average_query_time_customer1(1e8)

#### Assignment 2a

We now have two objective functions: `measure_average_query_time`, which is the average query time across all customers, and `measure_average_query_time_customer1`, which is the average query time for customer 1 only. How do we trade off between these two objectives? One way is to create one global objective function that is the weighted sum of these two individual objectives. Essentially, given a weight $w$, our global objective function is `measure_average_query_time + w * measure_average_query_time_customer1`. Implement this weighted objective below, while keeping in mind the total cache size constraint.

In [ ]:
### Assignment 2a ###
def create_weighted_objective_function(weight):
    def weighted_objective(cache_sizes):
        # Insert code here that returns a weighted global objective
    
    return weighted_objective

Intuitively, as the weight $w$ increases, the optimizer places increasing importance on minimizing customer 1's average query time, and the setting of $x$ found by the optimizer should result in a lower average query time for customer 1, although at the expense of having a higher overall average query time. Conceptually, there should be an optimal weight $w^*$ that induces the optimizer to satisfy customer 1's constraint while still maintaining a low overall average query time.

We can set this up as a two-level Bayesian optimization problem. In the outer level, we will optimize the weight $w$. In the inner level, we will optimize the cache sizes $x$ using the `weighted_objective` function you wrote above.

Hint: the objective function value for a weight that satisfies the constraint should not simply be 0.

In [ ]:
memo = {}  # Store the result of the inner-level BO for every iteration of the outer BO

def outer_objective(weight):
    weight = weight[0]  # the weight argument is a length-1 list, so we unwrap it
    weighted_objective = create_weighted_objective_function(weight)
    res = skopt.gp_minimize(
        weighted_objective,
        [(0, int(1e9))] * 10,
        n_calls=10,  # You can vary this if you want
    )
    memo[weight] = res.x
    # Insert code here that returns an objective value that induces the optimizer to find weights that satisfy customer 1's constraint.

Note that there is some randomness in Bayesian optimization so if you don't satisfy the constraint the first time you run, try running again (or increasing the number of calls `n_calls`) to see if you were just unlucky the first time. However, you should rarely violate the constraint two runs in a row.

In [ ]:
memo = {}  # Store the result of the inner-level BO for every iteration of the outer BO

res = skopt.gp_minimize(
    outer_objective,
    [(0.1, 10.)],
    n_calls=10,  # You can vary this if you want
)

best_cache_sizes = compute_scaled_cache_sizes(memo[res.x[0]])
print(f"Best weight: {res.x}, avg query time achieved: {measure_average_query_time(best_cache_sizes)}")
if int(sum(best_cache_sizes)) > 1e9:
    print("Oh no! Total cache size constraint violated. Something went wrong.")
if measure_average_query_time_customer1(best_cache_sizes[0]) > desired_cust1_avg_time:
    print("Oh no! Customer 1 average query time constraint violated. Something went wrong.")

#### Assignment 2b

Let's say that you know that the relationship between average query time and cache size for any individual customer is a monotonic function: as you increase cache size, average query time is non-increasing. Given this extra assumption, can you think of an even simpler way to minimize overall average query time while satisfying customer 1's query time constraint? You don't need to code anything here. Just describe the more efficient method:

Your text here

# Assignment 3: Time-based budgets

So far, we've set up the Bayesian optimizer by giving it a budget for the number of calls to the objective function that it's allowed to make (using the `n_calls` parameter). However, in practice what we really want is to give the Bayesian optimizer a time budget, e.g., "I'll give you 1 hour to find the best setting of $x$". If each call to the objective function takes the same amount of time, then a time budget is more or less equivalent to a function call budget. But in practice, calls to the objective function can take different amounts of time.

In our scenario, the objective function sets $x$, waits for a while, and measures the average query time. How long should we wait? If we wait for a short period of time, the measurement might be noisy, but we also don't want to wait for too long because that time might be better spent measuring a different setting of $x$.

For this assignment, we provide you with a function `create_cache_configuration`. It takes a single argument, which is the vector $x$ (represented as a Python list), and outputs a `CacheConfiguration` object which has a method `measure_average_query_time`. Every call to this method will wait for one additional unit of time and measure the average query time observed so far for that cache configuration. As a result, every successive call to `measure_average_query_time` returns decreasingly noisy measurements. If you execute the code below, you will notice that as we spend more time units, the measurement becomes more and more stable.

In [ ]:
with open("create_cache_configuration.bin", "rb") as f:
    create_cache_configuration = cloudpickle.loads(f.read())

In [ ]:
cache_config = create_cache_configuration([1e8] * 10)
for i in range(10):
    print(f"Time {i + 1} measurement: {cache_config.measure_average_query_time()}")

Suppose that you are given a time budget of 500 to find the best setting for $x$.

In [ ]:
time_budget = 500

How do you decide how many time units to spend when measuring each setting for $x$? One very simple strategy is to always measure for 10 timesteps. Here's an example for how to do that.

In [ ]:
# In practice, global variables should be avoided, but for simplicity we're going to have
# a global variable that measures the total time used. This must be reset before each trial
# of the Bayesian optimizer.
total_time_used = 0

# Simple strategy: always measure for 10 timesteps.
def simple_time_aware_objective(cache_sizes):
    global total_time_used
    cache_config = create_cache_configuration(compute_scaled_cache_sizes(cache_sizes))
    for _ in range(10):
        avg_query_time = cache_config.measure_average_query_time()
        total_time_used += 1
    return avg_query_time

In [ ]:
# We use this callback function with scikit-optimize to force the Bayesian optimizer to
# stop when the time budget is exceeded.
def callback(res):
    if total_time_used > time_budget:
        return True

In [ ]:
total_time_used = 0
res = skopt.gp_minimize(
    simple_time_aware_objective,
    [(0, int(1e9))] * 10,
    n_calls=500,  # You can vary this if you want
    callback=callback,
)
best_cache_sizes = compute_scaled_cache_sizes(res.x)
print(f"Total cache size: {sum(best_cache_sizes)}, avg query time achieved: {measure_average_query_time(best_cache_sizes)}")

However, there should be much smarter strategies for how to use our time budget. Your task is to create a smarter time-aware objective function. This assignment is meant to be more open-ended, and there isn't really any wrong solution.

In [ ]:
def time_aware_objective(cache_sizes):
    global total_time_used
    cache_config = create_cache_configuration(compute_scaled_cache_sizes(cache_sizes))
    # Insert your code here

Explain in text why you believe your time-aware objective function is better than the simple strategy:

Your text here

Aim for the average query time achieved by your time-aware objective to be better (i.e., lower) than that achieved by the simple strategy above, although it's not required as long as you gave a reason for your approach in the text above.

In [ ]:
total_time_used = 0
res = skopt.gp_minimize(
    time_aware_objective,
    [(0, int(1e9))] * 10,
    n_calls=500,
    callback=callback,
)
best_cache_sizes = compute_scaled_cache_sizes(res.x)
print(f"Total cache size: {sum(best_cache_sizes)}, avg query time achieved: {measure_average_query_time(best_cache_sizes)}")